In [1]:
import plotly.express as px
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import altair as alt
#alt.data_transformers.enable("vegafusion")
import math
from datetime import datetime, timedelta



#import csv
pd.set_option('display.max_rows', None)  # Display all rows

In [2]:
day = "06"
month = "10"

date = day + "_" + month #change date accordingly
date_year = "2023-" + month + "-" + day
dataset = "ble"

def process_file():
    #file_path = os.path.join(folder_path, f'rfid_{date}_{file_number}.csv')
    file_path = "all tests.csv"
    if os.path.exists(file_path):

        df = pd.read_csv(file_path, parse_dates=['Stamp'])
        if df.shape[1] == 6:
            df = df.drop(df.columns[0], axis=1)
            df = df.drop(df.columns[2], axis=1)
            df = df.drop(df.columns[2], axis=1)
            df = df.drop(df.columns[2], axis=1)
        
        df['Line'] = df['Line'].replace({'T012': '1', 'T013': '2'})
        df = df.sort_values(by='Stamp')
        df.to_csv(file_path, index = False)

    else:
        return pd.DataFrame

    return df

df_factory = process_file()

df_factory1 = df_factory[(df_factory['Line'] == 1)]
df_factory2 = df_factory[(df_factory['Line'] == 2)]

start_time = pd.to_datetime(date_year + " 09:00:00")
end_time = pd.to_datetime(date_year + " 18:00:00")
df_factory1 = df_factory1[((df_factory1['Stamp'] >= start_time) & (df_factory1['Stamp'] <= end_time))].reset_index(drop = True)
df_factory2 = df_factory2[((df_factory2['Stamp'] >= start_time) & (df_factory2['Stamp'] <= end_time))].reset_index(drop = True)
products1 = len(df_factory1)
products2 = len(df_factory2)
print(products1)
print(products2)

346
336


In [3]:
dates = ["2023-09-25", "2023-09-26", "2023-09-27", "2023-09-28", "2023-09-29", "2023-09-30", "2023-10-01", "2023-10-02", "2023-10-03", "2023-10-04", "2023-10-05", "2023-10-06", "2023-10-07", "2023-10-08", "2023-10-09", "2023-10-10", "2023-10-11"]
values = [304, 0, 333, 316, 0, 0, 0, 300, 313, 0, 0, 346, 0, 0, 295, 314, 270]
model = ['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'Y', 'Y']



colors = {'X':'dodgerblue', 'Y': 'orange'}

fig = px.bar(x=dates, y=values, color = model , color_discrete_map = colors)

fig.update_layout(
    title='Evolutions of number of products manufactured in production line monitored',
    xaxis_title='Date',
    yaxis_title='Products manufactured',
    legend=dict(
        title='Model')
    )

fig.update_traces(texttemplate='%{y}')
fig.update_xaxes(showgrid=True) 
fig.update_layout(font=dict(size=14))
fig.update_layout(width=850, height = 450) 

fig.show()

fig.write_image("images/prod_man_all_days" + ".pdf")


In [4]:
base = -1
base_person = -1

"""
if date == "11_10":
    base = 1
elif date == "13_10":
    base = 2
elif date == "24_08":
    base = 0
    base_person = 1
else:
    base = 0

if base == 1 or base ==2:
    folder_path = dataset + '_base/'
elif base_person == 1:
    folder_path = dataset + '_base_person/'
        """
folder_path = dataset +'_'+ date + '/'

def process_file(file_number, date):
    if date == '24_08':
        folder_path = dataset + '_base_person/'
    else:
        folder_path = dataset +'_'+ date + '/'
    file_path = os.path.join(folder_path, f'{dataset}_{date}_{file_number}.csv')
    print(file_path)
    if os.path.exists(file_path):
        df = pd.read_csv(file_path, parse_dates=['ts'])

        # Remove first empty column if it exists
        if df.shape[1] == 4:

            df = df.drop(df.columns[0], axis=1)
        # Keep entries with 'gabarit' equal to 1 or 2
        df_filt = df[(df['id'] == 2) | (df['id'] == 1)]
        df_filt = df_filt[(df_filt['rssi'] != 0)]

        # Write the updated DataFrame back to the CSV file
        df.to_csv(file_path, index = False)
        # Reset the index of the DataFrame
        df_filt = df_filt.reset_index(drop = True)

        return df_filt
    else:
        return pd.DataFrame

df1 = process_file(2,date)
df2 = process_file(1,date)
df3 = process_file(3,date)


ble_06_10/ble_06_10_2.csv
ble_06_10/ble_06_10_1.csv
ble_06_10/ble_06_10_3.csv


In [5]:
def plot_ble(df,station):
    # Plotly line graph
    color_discrete= {'1': 'tomato', '2': 'conrflowerblue'}

    fig = px.line(df, x='ts', y='rssi', color='id', color_discrete_map=color_discrete,
                  title=f'Line Graph for Station {station}')
    fig.update_layout(xaxis_title='Timestamp', yaxis_title='RSSI', legend_title='ID')
    fig.show()

plots = 0
if plots:
    plot_ble(df1,station =1)
    plot_ble(df2, station =2)
    plot_ble(df3, station = 3)



In [6]:
def analyse_df(df1, df2, df3, beacon):
    # Filter DataFrames for the given id value
    df1_id = df1[(df1['id'] == beacon)].reset_index(drop=True)
    df2_id = df2[(df2['id'] == beacon)].reset_index(drop=True)
    df3_id = df3[(df3['id'] == beacon)].reset_index(drop=True)


    # Combine dataframes with an outer merge
    merged_df = df1_id.merge(df2_id, on='ts', how='outer',suffixes = ['1','2']).merge(df3_id, on='ts', how='outer',suffixes = ['','3'])

    merged_df.drop(columns=['id1', 'id2', 'id'], inplace=True)
    merged_df.rename(columns={'rssi': 'rssi3'}, inplace=True)

    merged_df = merged_df.sort_values(by='ts')

    merged_df = merged_df.ffill()
    merged_df = merged_df.reset_index(drop=True)
    

    return merged_df


df_rssi1 = analyse_df(df1,df2,df3, beacon = 1) #beacon 1
df_rssi2 = analyse_df(df1,df2,df3, beacon = 2) #beacon 2

#print(df_rssi2)


In [7]:
# FIND BASE VALUES
def find_ref_ble_1_2(df_rssi1, df_rssi2):

    df_rssi1 = df_rssi1[(df_rssi1['rssi1'] != -100) & (df_rssi1['rssi2'] != -100) & (df_rssi1['rssi3'] != -100)]
    df_rssi2 = df_rssi2[(df_rssi2['rssi1'] != -100) & (df_rssi2['rssi2'] != -100) & (df_rssi2['rssi3'] != -100)]

    ref_values = np.zeros((3,3))
    #############################################################
    #                              station 1 station 2 station 3
    #beacon placed in station 1     X         X         X
    #beacon placed in station 2     X         X         X
    #beacon placed in station 3     X         X         X

    #beacon 2 placed in beggining (1)
    #beacon 1 placed in end on production line (2)
    med1_1 = df_rssi2['rssi1'].median()
    av1_1 = df_rssi2['rssi1'].mean()

    med2_1 = df_rssi2['rssi2'].median()
    av2_1 = df_rssi2['rssi2'].mean()

    med3_1 = df_rssi2['rssi3'].median()
    av_3_1 = df_rssi2['rssi3'].mean()

    print(med1_1, med2_1, med3_1)
    ref_values[0,:] = [av1_1,av2_1,av_3_1]

    med1_2 = df_rssi1['rssi1'].median()
    av1_2 = df_rssi1['rssi1'].mean()

    med2_2 = df_rssi1['rssi2'].median()
    av2_2 = df_rssi1['rssi2'].mean()

    med3_2 = df_rssi1['rssi3'].median()
    av_3_2 = df_rssi1['rssi3'].mean()

    #print(med1_2, med2_2, med3_2)
    ref_values[1,:] = [av1_2,av2_2,av_3_2]


    return ref_values

def find_ref_3(df_rssi1):
    df_rssi1 = df_rssi1[(df_rssi1['rssi1'] != -100) & (df_rssi1['rssi2'] != -100) & (df_rssi1['rssi3'] != -100)]

    # Filter DataFrame based on the time range
    med1_1 = df_rssi1['rssi1'].median()
    av1_1 = df_rssi1['rssi1'].mean()

    med2_1 = df_rssi1['rssi2'].median()
    av2_1 = df_rssi1['rssi2'].mean()

    med3_1 = df_rssi1['rssi3'].median()
    av_3_1 = df_rssi1['rssi3'].mean()

    #print(med1_1, med2_1, med3_1)
    ref_values[2,:] = [av1_1,av2_1,av_3_1]

    return ref_values

#if date == '11_10':
#    ref_values = find_ref_ble_1_2(df_rssi1, df_rssi2)
#    print(ref_values)
#elif date == '13_10':
#    ref_values = find_ref_3(df_rssi1)
#    print(ref_values)
#else:
ref_values=np.array(
[[-66.44549266, -74.26545582, -73.88842253],
 [-60.86128482, -43.43985976, -65.93448369],
 [-68.73783186, -56.43292456, -61.49354318]])




In [8]:
def state_df_person(df):
    #1 fim 18:02:00 -> 18:06:00
    #2 inicio 17:57:30 -> 18:01:00
    #3 monta cargas 18:07:00 -> 18:10:00
    df['real station'] = 0.0
    
    # Define the time frame
    start_time1 = pd.to_datetime('2023-08-24 17:57:35')
    end_time1 = pd.to_datetime('2023-08-24 18:00:33')
    start_time2 = pd.to_datetime('2023-08-24 18:02:00')
    end_time2 = pd.to_datetime('2023-08-24 18:05:20')
    start_time3 = pd.to_datetime('2023-08-24 18:07:05')
    end_time3 = pd.to_datetime('2023-08-24 18:10:00')

    df = df[((df['ts'] >= start_time1) & (df['ts'] <= end_time3))].reset_index(drop=True)
    df.loc[(df['ts'] >= start_time1) & (df['ts'] <= end_time1), 'real station'] = 1
    df.loc[(df['ts'] > end_time1) & (df['ts'] < start_time2), 'real station'] = 0.5
    df.loc[(df['ts'] >= start_time2) & (df['ts'] <= end_time2), 'real station'] = 2
    df.loc[(df['ts'] > end_time2) & (df['ts'] < start_time3), 'real station'] = 0.5
    df.loc[(df['ts'] >= start_time3) & (df['ts'] <= end_time3), 'real station'] = 3
    df.loc[(df['ts'] > end_time3)] = 0.5


    return df


if date == "24_08":
    #print(df_rssi2)
    df_rssi2 = state_df_person(df_rssi2)

if date == '11_10':
    df_rssi2['real station'] = 1
    #df_rssi1['real station'] = 2
    
if date == '13_10':
    df_rssi2['real station'] = 3


print(ref_values)


[[-66.44549266 -74.26545582 -73.88842253]
 [-60.86128482 -43.43985976 -65.93448369]
 [-68.73783186 -56.43292456 -61.49354318]]


In [9]:
confidence = 0.05
transition_station = [1.5,2.5]

def calculate_rms(rssi, rssi_ref):
    # Calculate RMS difference for each receiver
    rms_diff = np.sqrt(((rssi) - rssi_ref)**2)
    if math.isnan(rssi) or rssi == -100:
        rms_diff = 0

    return rms_diff

def define_weights(df_rssi, ref_values, iterations):
    # Create a DataFrame to store the results of different weight combinations
    results = pd.DataFrame(columns=['weight_station1', 'weight_station2', 'weight_station3', 'weight_rssi'])
    best_mean_accuracy  = 0 
    best_weights = None 

    # Iterate through weight combinations
    for w_station1 in np.linspace(0, 1, iterations):
        for w_station2 in np.linspace(0, 1, iterations):
            for w_station3 in np.linspace(0, 1, iterations):
                for w_rssi in np.linspace(0, 1, iterations):

                    if w_station1 + w_station2 + w_station3 + w_rssi == 1:
              
                        #print("weights:", w_station1, w_station2, w_station3, w_rssi)
                        accuracies = []
                        for index, row in df_rssi.iterrows():
                            if (row['real station'] == 0) | (row['real station']%1 != 0):
                                continue  # Skip rows with real station 0 (not defined)

                            # Calculate weighted RMS values for each station using reference values
                            rms_station = np.zeros(3)

                            for station in range(3):
                                rssi_values = [row['rssi1'], row['rssi2'], row['rssi3']]
                                
                                rms_station[station] = (w_station1 * calculate_rms(rssi_values[0], ref_values[station, 0]) + 
                                                        w_station2 * calculate_rms(rssi_values[1], ref_values[station, 1]) +
                                                        w_station3 * calculate_rms(rssi_values[2], ref_values[station, 2]))
                                
                            # Determine the station with the minimum weighted RMS value
                            station_ref = np.argmin(rms_station) + 1

                            # Calculate the station determined by the max RSSI
                            station_rssi = np.argmax([row['rssi1'], row['rssi2'], row['rssi3']]) + 1

                            # Calculate the weighted combination of station predictions
                            station_pred = (w_station1 + w_station2 + w_station3) * station_ref + w_rssi * station_rssi
                            
                            if abs(station_pred - round(station_pred)) <= confidence:
                                station_pred = round(station_pred)
                            else:
                                station_pred = 0
                                #= min(transition_station, key=lambda x: abs(x - station_pred)) #finds the closeste prediction state 
                            if station_pred == row['real station']:
                                accuracies.append(1)
                            #elif ((station_pred == 0) & (row['real station']%1 != 0)):
                            #    accuracies.append(1)
                            else:
                                accuracies.append(0)

                        mean_accuracy = sum(accuracies) / len(accuracies) *100
                        print(mean_accuracy)
                        print(w_station1,w_station2,w_station3)
                        if mean_accuracy >= best_mean_accuracy:
                            best_mean_accuracy = mean_accuracy
                            best_weights = {
                                'weight_station1': w_station1,
                                'weight_station2': w_station2,
                                'weight_station3': w_station3,
                                'weight_rssi': w_rssi,
                            }

                    else:
                        continue
    print("Accuracy", best_mean_accuracy)
    print("Best weights",best_weights)
    return best_weights


def classify_points(df_rssi, best_weights, ref_values):
    confidence = 0.15

    for index, row in df_rssi.iterrows():
        # Calculate weighted RMS values for each station using reference values
        
        rms_station = np.zeros(3)

        for station in range(3):
            rms_station[station] = (
                best_weights['weight_station1'] * calculate_rms(row['rssi1'], ref_values[station, 0]) +
                best_weights['weight_station2'] * calculate_rms(row['rssi2'], ref_values[station, 1]) +
                best_weights['weight_station3'] * calculate_rms(row['rssi3'], ref_values[station, 2])
            )

        # Determine the station with the minimum weighted RMS value
        station_ref = np.argmin(rms_station) + 1 # 1 2 or 3
        
        station_rssi = np.argmax([row['rssi1'], row['rssi2'], row['rssi3']]) + 1
        station_rssi

        station_pred = station_rssi * best_weights['weight_rssi'] + station_ref * (1 - best_weights['weight_rssi'])
        #CONFIANCA estado 1-2 2-3

        if abs(station_pred - round(station_pred)) <= confidence:
            station_pred = round(station_pred)
        else:
            station_pred = 0 # transition state
            #min(transition_station, key=lambda x: abs(x - station_pred))

        df_rssi.loc[index, 'station'] = station_pred
     
    return df_rssi

def calculate_accuracy(df_rssi):
    accuracy = np.zeros(4)  # Initialize an array to store accuracy values for each station
    all_stations = np.array([0, 1, 2, 3])

    for station in all_stations:
        total_points = 0
        total_correct = 0

        for index, row in df_rssi.iterrows():
            if row['real station'] == 0:
                continue  # Skip rows with real station 0 (not defined)

            if row['real station'] == station:
                total_points += 1
                if (row['station'] == row['real station']) | ((station == 0) & (row['real station']%1 != 0)):
                    total_correct += 1


        # Calculate accuracy for the current station
        index = np.where(all_stations == station)[0]
        accuracy[index] = total_correct / total_points * 100 if total_points > 0 else 0
    print("Accuracy: ",accuracy)
    return #accuracy


if date == '24_08':
    best_weights = define_weights(df_rssi2, ref_values, iterations=11)
    df_rssi2 = classify_points(df_rssi2,best_weights,ref_values)
    #display(df_rssi2)
    calculate_accuracy(df_rssi2)
else:
    best_weights = {
    'weight_station1': 0.0,
    'weight_station2': 0.1,
    'weight_station3': 0.0,
    'weight_rssi': 0.9
}

"""
if date == '11_10':
    #df_rssi2 = classify_points(df_rssi2,best_weights,ref_values)
    #calculate_accuracy(df_rssi2)

    df_rssi1 = classify_points(df_rssi1,best_weights,ref_values)
    calculate_accuracy(df_rssi1)
    #print(df_rssi1)

elif date == '13_10':
    df_rssi2 = classify_points(df_rssi2,best_weights,ref_values)
    calculate_accuracy(df_rssi2)
"""

print("Beacon 1")
df_rssi1 = classify_points(df_rssi1,best_weights,ref_values)
#display(df_rssi1.loc[:20])
#display(df_rssi1)

print("Beacon 2")
df_rssi2 = classify_points(df_rssi2,best_weights,ref_values)


Beacon 1
Beacon 2


In [10]:
def remove_outlier_transition(df):
    last_value = -1
    last_index = 0
    for i in range(len(df)):
        if ((df.loc[i,'station'] % 1 == 0) & 
            (df.loc[i,'station'] == last_value) & 
            (i > last_index + 1)):
            print("last_index", last_index)
            print("last_value", last_value)
            print("i", i)
            for j in range(last_index+1,i):
                df.loc[j,'station'] = last_value
            last_value = df.loc[i,'station']
            last_index = i
        else:
            continue
        display(df)
    return df

    
def remove_outliers(df):
    # Iterate through the DataFrame and update values
    for i in range(1, len(df) - 1):
        if df.at[i - 1, 'station'] == df.at[i + 1, 'station'] and df.at[i, 'station'] != df.at[i - 1, 'station']:
            df.at[i, 'station'] = df.at[i - 1, 'station']

    return df

def  remove_outliers2(df):
    # Iterate through the DataFrame and update values
    for i in range(1, len(df) - 2):
        if df.at[i - 1, 'station'] == df.at[i + 2, 'station'] and df.at[i, 'station'] == df.at[i + 1, 'station'] and df.at[i, 'station'] != df.at[i - 1, 'station']:
            df.at[i, 'station'] = df.at[i - 1, 'station']
            df.at[i + 1, 'station'] = df.at[i - 1, 'station']

    return df

def remove_outliers3(df):
    # Iterate through the DataFrame and update values
    for i in range(1, len(df) - 3):
        if (
            df.at[i - 1, 'station'] == df.at[i + 3, 'station']
            and df.at[i, 'station'] == df.at[i + 1, 'station']
            and df.at[i, 'station'] == df.at[i + 2, 'station']
            and df.at[i, 'station'] != df.at[i - 1, 'station']
        ):
            df.at[i, 'station'] = df.at[i - 1, 'station']
            df.at[i + 1, 'station'] = df.at[i - 1, 'station']
            df.at[i + 2, 'station'] = df.at[i - 1, 'station']

    return df

def remove_outliers4(df):
    # Iterate through the DataFrame and update values
    for i in range(1, len(df) - 4):
        if (
            df.at[i - 1, 'station'] == df.at[i + 4, 'station']
            and df.at[i, 'station'] == df.at[i + 1, 'station']
            and df.at[i, 'station'] == df.at[i + 2, 'station']
            and df.at[i, 'station'] == df.at[i + 3, 'station']
            and df.at[i, 'station'] != df.at[i - 1, 'station']
        ):
            df.at[i, 'station'] = df.at[i - 1, 'station']
            df.at[i + 1, 'station'] = df.at[i - 1, 'station']
            df.at[i + 2, 'station'] = df.at[i - 1, 'station']
            df.at[i + 3, 'station'] = df.at[i - 1, 'station']

    return df

def remove_outliers5(df):
    # Iterate through the DataFrame and update values
    for i in range(1, len(df) - 5):
        if (
            df.at[i - 1, 'station'] == df.at[i + 5, 'station']
            and df.at[i, 'station'] == df.at[i + 1, 'station']
            and df.at[i, 'station'] == df.at[i + 2, 'station']
            and df.at[i, 'station'] == df.at[i + 3, 'station']
            and df.at[i, 'station'] == df.at[i + 4, 'station']
            and df.at[i, 'station'] != df.at[i - 1, 'station']
        ):
            df.at[i, 'station'] = df.at[i - 1, 'station']
            df.at[i + 1, 'station'] = df.at[i - 1, 'station']
            df.at[i + 2, 'station'] = df.at[i - 1, 'station']
            df.at[i + 3, 'station'] = df.at[i - 1, 'station']
            df.at[i + 4, 'station'] = df.at[i - 1, 'station']

    return df



#eliminiate_outliers(df_rssi1)
#print(df_rssi1)
#display(df_rssi2.loc[:22])
#calculate_accuracy(df_rssi2)

df_rssi2 = remove_outlier_transition(df_rssi2)
df_rssi2 = remove_outliers(df_rssi2)
df_rssi2 = remove_outliers2(df_rssi2)
df_rssi2 = remove_outliers3(df_rssi2)
df_rssi2 = remove_outliers4(df_rssi2)
df_rssi2 = remove_outliers3(df_rssi2)
df_rssi2 = remove_outliers2(df_rssi2)
df_rssi2 = remove_outliers(df_rssi2)


df_rssi1 = remove_outlier_transition(df_rssi1)
df_rssi1 = remove_outliers(df_rssi1)
df_rssi1 = remove_outliers2(df_rssi1)
df_rssi1 = remove_outliers3(df_rssi1)
df_rssi1 = remove_outliers4(df_rssi1)
df_rssi1 = remove_outliers3(df_rssi1)
df_rssi1 = remove_outliers2(df_rssi1)
df_rssi1 = remove_outliers(df_rssi1)



#calculate_accuracy(df_rssi1)
#calculate_accuracy(df_rssi2)
#[96.77419355 81.81818182 88.23529412]

In [11]:
def remove_breaks_from_rssi(df_rssi, df_factory):
    df_factory_ = df_factory.copy()
    # Calculate the time gaps in df_factory
    gap_threshold = pd.Timedelta(minutes=5)
    df_factory_['time_diff'] = df_factory_['Stamp'].diff()
    #df_factory_['time_diff'] = df_factory_['Stamp'] - df_factory_['Stamp'].shift(periods=1, fill_value=df_factory_['Stamp'].iloc[0])

    # Identify gaps longer than 5 minutes
    long_gaps = df_factory_[df_factory_['time_diff'] > gap_threshold]
    print(long_gaps)
    print()
    #last_ts = pd.to_datetime(date_year + " 09:00:00")
    # Remove data from df_rssi corresponding to long gaps in df_factory
    for index, row in long_gaps.iterrows():
        #print(last_ts)
        print(df_factory_.loc[index-1,'Stamp'])
        print()
        #df_rssi = df_rssi[(df_rssi['ts'] > df_factory_.loc[index-1,'Stamp']) & (df_rssi['ts'] < last_ts)]
        remove_breaks = ~((df_rssi['ts'] > df_factory_.loc[index-1,'Stamp']) & 
                         (df_rssi['ts'] < df_factory_.loc[index-1,'Stamp'] + row['time_diff']))
        # Apply the boolean mask to filter the rows in df_Rssi
        df_rssi = df_rssi[remove_breaks]
        #last_ts = df_factory_.loc[index-1,'Stamp'] + row['time_diff']

    return df_rssi

print("Beacon 1")
df_rssi1 = remove_breaks_from_rssi(df_rssi1, df_factory1)
print("Beacon 2")
df_rssi2 = remove_breaks_from_rssi(df_rssi2, df_factory2)
#print(df_rssi2)
#print(df_factory1)
#print(df_rssi1)

Beacon 1
                  Stamp  Line       time_diff
69  2023-10-06 10:47:08     1 0 days 00:16:40
172 2023-10-06 14:01:56     1 0 days 01:08:29
294 2023-10-06 16:49:53     1 0 days 00:17:02

2023-10-06 10:30:28

2023-10-06 12:53:27

2023-10-06 16:32:51

Beacon 2
                  Stamp  Line       time_diff
64  2023-10-06 10:46:13     2 0 days 00:17:03
168 2023-10-06 14:01:36     2 0 days 01:05:53
284 2023-10-06 16:49:27     2 0 days 00:18:02

2023-10-06 10:29:10

2023-10-06 12:55:43

2023-10-06 16:31:25



In [12]:

def time_in_station_ble(df_rssi):
    # Initialize time variables for each station
    time_in_1 = pd.to_timedelta('0 seconds')
    time_in_2 = pd.to_timedelta('0 seconds')
    time_in_3 = pd.to_timedelta('0 seconds')
    total_time = pd.to_timedelta('0 seconds')

    for index, row in df_rssi.iterrows():
        # Calculate time spent in each station based on station assignments
        if row['station'] == 1:
            if index == 0:
                continue
            if index - 1 in df_rssi.index :
            #    if df_rssi.loc[index - 1, 'station'] == 1:
                time_in_1 += df_rssi.loc[index, 'ts'] - df_rssi.loc[index - 1, 'ts']

        if row['station'] == 2:
            if index == 0:
                continue
            if index-1 in df_rssi.index:
            #    if df_rssi.loc[index - 1, 'station'] == 2:
                time_in_2 += df_rssi.loc[index, 'ts'] - df_rssi.loc[index - 1, 'ts']

        if row['station'] == 3:
            if index == 0:
                continue
            if index-1 in df_rssi.index:
            #    if df_rssi.loc[index - 1, 'station'] == 3:
                time_in_3 += df_rssi.loc[index, 'ts'] - df_rssi.loc[index - 1, 'ts']

    # Calculate total time and percentages for each station
    total_time = time_in_1 + time_in_2 + time_in_3
    perc1 = time_in_1 / total_time * 100
    perc1 = round(perc1, 2)
    perc2 = time_in_2 / total_time * 100
    perc2 = round(perc2, 2)
    perc3 = time_in_3 / total_time * 100
    perc3 = round(perc3, 2)
    #print("PREDICTED")
    print("time in each station", time_in_1,time_in_2,time_in_3)
    print("total time recorded:", time_in_1+time_in_2+time_in_3)
    print(f'station1: {perc1}%,    station2: {perc2}%,    station3: {perc3}%')
    return time_in_1,time_in_2,time_in_3

def time_in_real_station_ble(df_rssi):
    # Initialize time variables for each station
    time_in_1 = pd.to_timedelta('0 seconds')
    time_in_2 = pd.to_timedelta('0 seconds')
    time_in_3 = pd.to_timedelta('0 seconds')

    for index, row in df_rssi.iterrows():
        # Calculate time spent in each station based on station assignments
        if row['real station'] == 1:
            if index == 0:
                continue
            if index-1 in df_rssi.index:
                time_in_1 += df_rssi.loc[index, 'ts'] - df_rssi.loc[index - 1, 'ts']
        if row['real station'] == 2:
            if index == 0:
                continue
            if index-1 in df_rssi.index:
                time_in_2 += df_rssi.loc[index, 'ts'] - df_rssi.loc[index - 1, 'ts']

        if row['real station'] == 3:
            if index == 0:
                continue
            if index-1 in df_rssi.index:
                time_in_3 += df_rssi.loc[index, 'ts'] - df_rssi.loc[index - 1, 'ts']

    # Calculate total time and percentages for each station
    total_time = time_in_1 + time_in_2 + time_in_3
    perc1 = time_in_1 / total_time * 100
    perc1 = round(perc1, 2)
    perc2 = time_in_2 / total_time * 100
    perc2 = round(perc2, 2)
    perc3 = time_in_3 / total_time * 100
    perc3 = round(perc3, 2)
    print("REAL")
    print("time in each station", time_in_1,time_in_2,time_in_3)
    print("total time recorded:", time_in_1+time_in_2+time_in_3)
    print(f'station1: {perc1}%,    station2: {perc2}%,    station3: {perc3}%')
    return

if date == "24_08":
    start_time1 = pd.to_datetime('2023-08-24 17:57:35')
    end_time1 = pd.to_datetime('2023-08-24 18:00:33')
    start_time2 = pd.to_datetime('2023-08-24 18:02:00')
    end_time2 = pd.to_datetime('2023-08-24 18:05:20')
    start_time3 = pd.to_datetime('2023-08-24 18:07:05')
    end_time3 = pd.to_datetime('2023-08-24 18:10:00')

    df_rssi2 = df_rssi2[((df_rssi2['ts'] >= start_time1) & (df_rssi2['ts'] <= end_time3))]
    df_rssi2.loc[(df_rssi2['ts'] >= end_time1) & (df_rssi2['ts'] <= start_time2), 'station'] = 4
    df_rssi2.loc[(df_rssi2['ts'] >= end_time2) & (df_rssi2['ts'] <= start_time3), 'station'] = 4
    df_rssi2.loc[(df_rssi2['ts'] >= end_time3), 'station'] = 4


print("Beacon 1")
time_in_beacon1 =  time_in_station_ble(df_rssi1)
#display(df_rssi1)

print("Beacon 2")
time_in_beacon2 = time_in_station_ble(df_rssi2)
#time_in_beacon2_real = time_in_real_station_ble(df_rssi2)
#display(df_rssi2.loc[:100])
#display(df_rssi2)





Beacon 1
time in each station 0 days 00:17:16 0 days 05:38:31 0 days 00:40:54
total time recorded: 0 days 06:36:41
station1: 4.35%,    station2: 85.34%,    station3: 10.31%
Beacon 2
time in each station 0 days 00:30:43 0 days 04:52:07 0 days 00:47:01
total time recorded: 0 days 06:09:51
station1: 8.31%,    station2: 78.98%,    station3: 12.71%


In [13]:
def count_visits_station(df, visits,station):
    current_station = None
    visit_start_time = None
    station_visits1 = 0
    station_visits2 = 0
    station_visits3 = 0
    dwell_time_sum1 = 0
    dwell_time_sum2 = 0
    dwell_time_sum3 = 0

    # Iterate through the sorted data to identify station visits
    for index, row in df.iterrows():
        if row[station] != current_station:
            # New station visit
            if current_station is not None and visit_start_time is not None:
                dwell_time = (row['ts'] - visit_start_time).total_seconds()
                if dwell_time > 5:
                    # Check if the current_station is 1, 2, or 3 and increment the respective count and dwell time sum
                    if current_station == 1:
                        station_visits1 += 1
                        dwell_time_sum1 += dwell_time
                    elif current_station == 2:
                        station_visits2 += 1
                        dwell_time_sum2 += dwell_time
                    elif current_station == 3:
                        station_visits3 += 1
                        dwell_time_sum3 += dwell_time
            current_station = row[station]
            visit_start_time = row['ts']

    # Add the last station visit
    if current_station is not None and visit_start_time is not None:
        dwell_time = (df['ts'].iloc[-1] - visit_start_time).total_seconds()
        if dwell_time > 5:
            # Check if the current_station is 1, 2, or 3 and increment the respective count and dwell time sum
            if current_station == 1:
                station_visits1 += 1
                dwell_time_sum1 += dwell_time
            elif current_station == 2:
                station_visits2 += 1
                dwell_time_sum2 += dwell_time
            elif current_station == 3:
                station_visits3 += 1
                dwell_time_sum3 += dwell_time


    if visits == 1:
        print("Visit stations: ",station_visits1, station_visits2, station_visits3)
        return station_visits1, station_visits2, station_visits3
    else: 
        dwell_time_avg1 = round(dwell_time_sum1/station_visits1)
        dwell_time_avg2 = round(dwell_time_sum2/station_visits2)
        dwell_time_avg3 = round(dwell_time_sum3/station_visits3)
        dwell_time_avg1 = pd.Timedelta(seconds=dwell_time_avg1)
        dwell_time_avg2 = pd.Timedelta(seconds=dwell_time_avg2)
        dwell_time_avg3 = pd.Timedelta(seconds=dwell_time_avg3)

        print("Dwell times: ", dwell_time_avg1, dwell_time_avg2, dwell_time_avg3)
        return dwell_time_avg1, dwell_time_avg2, dwell_time_avg3


print("Beacon 1")
time_in_beacon1 =  time_in_station_ble(df_rssi1)
visits1 = count_visits_station(df_rssi1,1,station='station')
dwell1 = count_visits_station(df_rssi1,2,station='station')
#display(df_rssi1)

print("Beacon 2")
time_in_beacon2 = time_in_station_ble(df_rssi2)
visits2 = count_visits_station(df_rssi2,1,station='station')
dwell2 = count_visits_station(df_rssi2,2,station='station')


products1 = len(df_factory1)
products2 = len(df_factory2)
print(products1)
print(products2)

#products1 270,314,295,346,313
#products2 308,319,301,336,314


Beacon 1
time in each station 0 days 00:17:16 0 days 05:38:31 0 days 00:40:54
total time recorded: 0 days 06:36:41
station1: 4.35%,    station2: 85.34%,    station3: 10.31%
Visit stations:  38 180 86
Dwell times:  0 days 00:00:24 0 days 00:02:23 0 days 00:00:27
Beacon 2
time in each station 0 days 00:30:43 0 days 04:52:07 0 days 00:47:01
total time recorded: 0 days 06:09:51
station1: 8.31%,    station2: 78.98%,    station3: 12.71%
Visit stations:  77 232 90
Dwell times:  0 days 00:01:13 0 days 00:01:25 0 days 00:00:29
346
336


In [14]:
# Resampling with forward fill
def resample_df(df, freq):
    # Set the 'Timestamp' column as the DataFrame index
    duplicates = df[df.duplicated(subset='ts', keep=False)]
    for index, row in duplicates.iterrows():
        if index % 2 == 0:
            df.at[index, 'ts'] = row['ts'] + timedelta(seconds=1)
            
    df = df.sort_values(by='ts')

    df.set_index('ts', inplace=True)

    # Resample the DataFrame with a desired frequency (e.g., 10 minutes) and forward fill
    resampled_df = df.resample(freq).ffill()

    # Reset the index to have the timestamp as a column again
    resampled_df = resampled_df.reset_index()

    return resampled_df

#start_time = pd.to_datetime(date_year + " 17:58:00")
#end_time = pd.to_datetime(date_year + " 18:10:01")
start_time = pd.to_datetime(date_year + " 09:00:00")
start_lunch = pd.to_datetime(date_year + " 13:00:00")
end_lunch = pd.to_datetime(date_year + " 14:00:00")
end_time = pd.to_datetime(date_year + " 18:00:00")



#df_rssi1_ = resample_df(df_rssi1,freq = '5S')
if date == "24_08":
    start_time24_08 = pd.to_datetime(date_year + " 17:58:00")
    end_time24_08 = pd.to_datetime(date_year + " 18:10:01")

    df24_08_resample = resample_df(df_rssi2,freq = '1S')
    df24_08 = df24_08_resample[((df24_08_resample['ts'] >= start_time24_08) & (df24_08_resample['ts'] <= end_time24_08)) ]
    df24_08 = df24_08.reset_index(drop=True)




#morning
df1_morning_ = df_rssi1[((df_rssi1['ts'] >= start_time) & (df_rssi1['ts'] <= start_lunch))]
#afternoon
df1_afternoon_ = df_rssi1[((df_rssi1['ts'] >= end_lunch) & (df_rssi1['ts'] <= end_time))]



df1_morning = resample_df(df1_morning_,freq = '3S')
df1_afternoon = resample_df(df1_afternoon_,freq = '3S')
df1_morning = df1_morning.reset_index(drop=True)
df1_afternoon = df1_afternoon.reset_index(drop=True)
#morning
df2_morning_ = df_rssi2[((df_rssi2['ts'] >= start_time) & (df_rssi2['ts'] <= start_lunch))] 
#afternoon
df2_afternoon_ = df_rssi2[((df_rssi2['ts'] >= end_lunch) & (df_rssi2['ts'] <= end_time))]

df2_morning = resample_df(df2_morning_,freq = '3S')
df2_afternoon = resample_df(df2_afternoon_,freq = '3S')

df2_morning = df2_morning.reset_index(drop=True )
df2_afternoon = df2_afternoon.reset_index(drop=True )


df_factory1_morn = df_factory1[((df_factory1['Stamp'] >= start_time) & (df_factory1['Stamp'] <= start_lunch))] 
df_factory1_after = df_factory1[((df_factory1['Stamp'] >= end_lunch) & (df_factory1['Stamp'] <= end_time))]
df_factory1_morn = df_factory1_morn.reset_index(drop=True )
df_factory1_after = df_factory1_after.reset_index(drop=True )

df_factory2_morn = df_factory2[((df_factory2['Stamp'] >= start_time) & (df_factory2['Stamp'] <= start_lunch))] 
df_factory2_after = df_factory2[((df_factory2['Stamp'] >= end_lunch) & (df_factory2['Stamp'] <= end_time))]
df_factory2_morn = df_factory2_morn.reset_index(drop=True )
df_factory2_after = df_factory2_after.reset_index(drop=True )




In [15]:
columns = ['Date', 'Beacon', 'Time in Station1', 'Time in Station2','Time in Station2',
            'Station Counts1', 'Station Counts2', 'Station Counts3', 
            'Average Dwell Times1', 'Average Dwell Times2', 'Average Dwell Times3']
df_metrics = pd.DataFrame(columns=columns)

row_11_10_1 = {
        'Date': '2023-10-11',
        'Beacon': 1,
        'Time in Station1': 13.88,
        'Time in Station2': 71.36,
        'Time in Station3': 14.76,
        'Station Counts1': 49  ,
        'Station Counts2': 171 ,
        'Station Counts3': 104,
        'Average Dwell Times1': pd.to_timedelta('0 days 00:01:02'),
        'Average Dwell Times2': pd.to_timedelta('0 days 00:02:26'),
        'Average Dwell Times3': pd.to_timedelta('0 days 00:00:32')
    }

row_11_10_2 = {
        'Date': '2023-10-11',
        'Beacon': 2,
        'Time in Station1': 6.14,
        'Time in Station2': 56.45,
        'Time in Station3': 37.41,
        'Station Counts1': 55  ,
        'Station Counts2': 196 ,
        'Station Counts3': 183,
        'Average Dwell Times1': pd.to_timedelta('0 days 00:00:21'),
        'Average Dwell Times2': pd.to_timedelta('0 days 00:01:38'),
        'Average Dwell Times3': pd.to_timedelta('0 days 00:01:02')
    }

row_10_10_1 = {
        'Date': '2023-10-10',
        'Beacon': 1,
        'Time in Station1': 51.97,
        'Time in Station2': 34.62,
        'Time in Station3': 13.41,
        'Station Counts1': 154  ,
        'Station Counts2': 128 ,
        'Station Counts3': 110,
        'Average Dwell Times1': pd.to_timedelta('0 days 00:01:53'),
        'Average Dwell Times2': pd.to_timedelta('0 days 00:01:07'),
        'Average Dwell Times3': pd.to_timedelta('0 days 00:00:33')
    }

row_10_10_2 = {
        'Date': '2023-10-10',
        'Beacon': 2,
        'Time in Station1': 7.82,
        'Time in Station2': 54.95,
        'Time in Station3': 37.23,
        'Station Counts1': 93  ,
        'Station Counts2': 234 ,
        'Station Counts3': 211,
        'Average Dwell Times1': pd.to_timedelta('0 days 00:00:22'),
        'Average Dwell Times2': pd.to_timedelta('0 days 00:01:26'),
        'Average Dwell Times3': pd.to_timedelta('0 days 00:00:44')
    }





row_3_10_1 = {
        'Date': '2023-10-03',
        'Beacon': 1,
        'Time in Station1': 1.14,
        'Time in Station2': 91.26,
        'Time in Station3': 7.6,
        'Station Counts1': 10,
        'Station Counts2': 76,
        'Station Counts3': 56,
        'Average Dwell Times1': pd.to_timedelta('0 days 00:00:16'),
        'Average Dwell Times2': pd.to_timedelta('0 days 00:06:38'),
        'Average Dwell Times3': pd.to_timedelta('0 days 00:00:31')
    }

row_3_10_2 = {
        'Date': '2023-10-03',
        'Beacon': 2,
        'Time in Station1': 11.97,
        'Time in Station2': 71.25,
        'Time in Station3': 16.79,
        'Station Counts1': 105 ,
        'Station Counts2': 187,
        'Station Counts3': 128,
        'Average Dwell Times1': pd.to_timedelta('0 days 00:00:27'),
        'Average Dwell Times2': pd.to_timedelta('0 days 00:02:12'),
        'Average Dwell Times3': pd.to_timedelta('0 days 00:00:31')
    }

row_6_10_1 = {
        'Date': '2023-10-06',
        'Beacon': 1,
        'Time in Station1': 5.25,
        'Time in Station2': 75.96,
        'Time in Station3': 18.8,
        'Station Counts1': 52 ,
        'Station Counts2': 171 ,
        'Station Counts3': 121,
        'Average Dwell Times1': pd.to_timedelta('0 days 00:00:24'),
        'Average Dwell Times2': pd.to_timedelta('0 days 00:02:32'),
        'Average Dwell Times3': pd.to_timedelta('0 days 00:00:39')
    }
row_6_10_2 = {
        'Date': '2023-10-06',
        'Beacon': 2,
        'Time in Station1': 10.76,
        'Time in Station2': 67.37,
        'Time in Station3': 21.887,
        'Station Counts1': 115 ,
        'Station Counts2': 229 ,
        'Station Counts3': 163,
        'Average Dwell Times1': pd.to_timedelta('0 days 00:00:22'),
        'Average Dwell Times2': pd.to_timedelta('0 days 00:01:42'),
        'Average Dwell Times3': pd.to_timedelta('0 days 00:00:33')
    }

row_9_10_1 = {
        'Date': '2023-10-09',
        'Beacon': 1,
        'Time in Station1': 3.92,
        'Time in Station2': 78.47,
        'Time in Station3': 17.61,
        'Station Counts1': 19  ,
        'Station Counts2': 126 ,
        'Station Counts3': 110,
        'Average Dwell Times1': pd.to_timedelta('0 days 00:01:38'),
        'Average Dwell Times2': pd.to_timedelta('0 days 00:03:22'),
        'Average Dwell Times3': pd.to_timedelta('0 days 00:00:42')
    }

row_9_10_2 = {
        'Date': '2023-10-09',
        'Beacon': 2,
        'Time in Station1': 4.23,
        'Time in Station2': 63.19,
        'Time in Station3': 32.57,
        'Station Counts1': 43  ,
        'Station Counts2': 168 ,
        'Station Counts3': 154,
        'Average Dwell Times1': pd.to_timedelta('0 days 00:00:20'),
        'Average Dwell Times2': pd.to_timedelta('0 days 00:02:02'),
        'Average Dwell Times3': pd.to_timedelta('0 days 00:01:09')
    }

df_3_10_1 = pd.DataFrame([row_3_10_1])
df_3_10_2 = pd.DataFrame([row_3_10_2])
df_6_10_1 = pd.DataFrame([row_6_10_1])
df_6_10_2 = pd.DataFrame([row_6_10_2])
df_9_10_1 = pd.DataFrame([row_9_10_1])
df_9_10_2 = pd.DataFrame([row_9_10_2])
df_10_10_1 = pd.DataFrame([row_10_10_1])
df_10_10_2 = pd.DataFrame([row_10_10_2])
df_11_10_1 = pd.DataFrame([row_11_10_1])
df_11_10_2 = pd.DataFrame([row_11_10_2])

df_metrics = pd.concat([df_3_10_1,df_3_10_2, df_6_10_1, df_6_10_2, df_9_10_1, df_9_10_2, df_10_10_1, df_10_10_2,df_11_10_1, df_11_10_2], ignore_index=True)

display(df_metrics)

numeric_columns = ["Time in Station1", "Time in Station2", "Time in Station3", "Station Counts1", "Station Counts2", "Station Counts3"]
# Convert the specified numeric columns to float
df_metrics[numeric_columns] = df_metrics[numeric_columns].apply(pd.to_numeric, errors='coerce')
df_metrics_mean = df_metrics.drop(columns=["Date","Average Dwell Times1","Average Dwell Times2","Average Dwell Times3"])
means_per_beacon = df_metrics_mean.groupby("Beacon").mean()
print(means_per_beacon)


dwell_columns = ["Average Dwell Times1", "Average Dwell Times2", "Average Dwell Times3"]
grouped = df_metrics.groupby('Beacon')
average_dwell_times = grouped[dwell_columns].mean()

# Convert the mean values to Timedelta
for column in dwell_columns:
    average_dwell_times[column] = average_dwell_times[column].apply(lambda x: pd.Timedelta(seconds=x.total_seconds()))

print(average_dwell_times)
#print(average_dwell)

print("Products by 1",len(df_factory1))
print("Products by 2",len(df_factory2))


,Date,Beacon,Time in Station1,Time in Station2,Time in Station3,Station Counts1,Station Counts2,Station Counts3,Average Dwell Times1,Average Dwell Times2,Average Dwell Times3
0,2023-10-03,1,1.14,91.26,7.600,10,76,56,0 days 00:00:16,0 days 00:06:38,0 days 00:00:31
1,2023-10-03,2,11.97,71.25,16.790,105,187,128,0 days 00:00:27,0 days 00:02:12,0 days 00:00:31
2,2023-10-06,1,5.25,75.96,18.800,52,171,121,0 days 00:00:24,0 days 00:02:32,0 days 00:00:39
3,2023-10-06,2,10.76,67.37,21.887,115,229,163,0 days 00:00:22,0 days 00:01:42,0 days 00:00:33
4,2023-10-09,1,3.92,78.47,17.610,19,126,110,0 days 00:01:38,0 days 00:03:22,0 days 00:00:42
5,2023-10-09,2,4.23,63.19,32.570,43,168,154,0 days 00:00:20,0 days 00:02:02,0 days 00:01:09
6,2023-10-10,1,51.97,34.62,13.410,154,128,110,0 days 00:01:53,0 days 00:01:07,0 days 00:00:33
7,2023-10-10,2,7.82,54.95,37.230,93,234,211,0 days 00:00:22,0 days 00:01:26,0 days 00:00:44
8,2023-10-11,1,13.88,71.36,14.760,49,171,104,0 days 00:01:02,0 days 00:02:26,0 days 00:00:32
9,2023-10-11,2,6.14,56.45,37.410,55,196,183,0 days 00:00:21,0 days 00:01:38,0 days 00:01:02


        Time in Station1  Time in Station2  Time in Station3  Station Counts1  \
Beacon                                                                          
1                 15.232            70.334           14.4360             56.8   
2                  8.184            62.642           29.1774             82.2   

        Station Counts2  Station Counts3  
Beacon                                    
1                 134.4            100.2  
2                 202.8            167.8  
         Average Dwell Times1 Average Dwell Times2   Average Dwell Times3
Beacon                                                                   
1      0 days 00:01:02.600000      0 days 00:03:13 0 days 00:00:35.400000
2      0 days 00:00:22.400000      0 days 00:01:48 0 days 00:00:47.800000
Products by 1 346
Products by 2 336


In [16]:
start_time = pd.to_datetime(date_year + " 09:00:00")
start_lunch = pd.to_datetime(date_year + " 13:00:00")
end_lunch = pd.to_datetime(date_year + " 14:00:00")
end_time = pd.to_datetime(date_year + " 18:00:00")

df1_morning = df_rssi1[((df_rssi1['ts'] >= start_time) & (df_rssi1['ts'] <= start_lunch))]
df1_afternoon = df_rssi1[((df_rssi1['ts'] >= end_lunch) & (df_rssi1['ts'] <= end_time))]

df1_morning = resample_df(df1_morning,freq = '3S')
df1_afternoon = resample_df(df1_afternoon,freq = '3S')

df1_morning = df1_morning.reset_index(drop=True)
df1_afternoon = df1_afternoon.reset_index(drop=True)


df2_morning = df_rssi2[((df_rssi2['ts'] >= start_time) & (df_rssi2['ts'] <= start_lunch))] 
df2_afternoon = df_rssi2[((df_rssi2['ts'] >= end_lunch) & (df_rssi2['ts'] <= end_time))]

df2_morning = resample_df(df2_morning,freq = '3S')
df2_afternoon = resample_df(df2_afternoon,freq = '3S')
df2_morning = df2_morning.reset_index(drop=True)
df2_afternoon = df2_afternoon.reset_index(drop=True )


df_factory1_morn = df_factory1[((df_factory1['Stamp'] >= start_time) & (df_factory1['Stamp'] <= start_lunch))] 
df_factory1_after = df_factory1[((df_factory1['Stamp'] >= end_lunch) & (df_factory1['Stamp'] <= end_time))]
df_factory1_morn = df_factory1_morn.reset_index(drop=True )
df_factory1_after = df_factory1_after.reset_index(drop=True )


df_factory2_morn = df_factory2[((df_factory2['Stamp'] >= start_time) & (df_factory2['Stamp'] <= start_lunch))] 
df_factory2_after = df_factory2[((df_factory2['Stamp'] >= end_lunch) & (df_factory2['Stamp'] <= end_time))]
df_factory2_morn = df_factory2_morn.reset_index(drop=True )
df_factory2_after = df_factory2_after.reset_index(drop=True )

In [17]:

def strip_plot(df,date, beacon, timeofday):
    df['ts'] = df['ts'].astype(str)  
    y_scale = alt.Scale(domain=[0, 1.0, 2.0, 3.0])
    color_real = alt.Color('real station:N', scale=alt.Scale(
        domain=[0, 1.0, 2.0, 3.0], range=['gray', 'cornflowerblue', 'darkorange','violet'])
            , legend=alt.Legend(title='Station'))
    color = alt.Color('station:N', scale=alt.Scale(
        domain=[0, 1.0, 2.0, 3.0], range=['gray', 'cornflowerblue', 'darkorange','violet'])
            , legend=alt.Legend(title='Station'))

    # Custom ticks
    #custom_x_ticks = [date_year + '09:00:00', date_year + '10:00:00', date_year + '11:00:00', date_year + '12:00:00', 
    #                  date_year + '13:00:00', date_year + '15:00:00', date_year + '16:00:00', date_year + '17:00:00', date_year + '18:00:00']    #1 inicio 17:57:30 -> 18:01:00
    
    #custom_x_ticks = ['2023-08-24 17:58:00', '2023-08-24 18:00:00', '2023-08-24 18:02:00', '2023-08-24 18:04:00', '2023-08-24 18:06:00', '2023-08-24 18:08:00', '2023-08-24 18:10:00']    #1 inicio 17:57:30 -> 18:01:00
    if timeofday == 'morning':
        x_scale = alt.Scale(domain=[date_year + ' 09:00:00', date_year + ' 13:00:00'])
        #x_scale = alt.Scale(domain=[pd.date_range(start=date_year + ' 09:00:00', end=date_year + ' 13:00:00', freq='30T')])
    else: 
        x_scale = alt.Scale(domain=[date_year + ' 14:00:00', date_year + ' 18:00:00'])

    chart = alt.Chart(df).mark_tick().encode(
        x=alt.X('ts:T', title='Timestamp',scale=x_scale,axis=alt.Axis(format='%H:%M')), #,axis=alt.Axis(values=custom_x_ticks)), # axis=alt.Axis(values=custom_x_ticks)),
        y=alt.Y('station:O', title='Station', scale=y_scale), 
        color=color  # Apply the color encoding
    ).properties(
        title=f'Classified station of beacon {beacon} in the {timeofday} ({date})',#,#{beacon}',
        width=400,  
        height=125 
    ).interactive()

    return chart

def strip_factory_plot(df,date,line, timeofday):
    df['Stamp'] = df['Stamp'].astype(str)  
    if line == 1:
        y_scale = alt.Scale(domain=[1])
    else:
        y_scale = alt.Scale(domain=[2])

    if timeofday == 'morning':
        x_scale = alt.Scale(domain=[date_year + ' 09:00:00', date_year + ' 13:00:00'])
    else: 
        x_scale = alt.Scale(domain=[date_year + ' 14:00:00', date_year + ' 18:00:00'])
    
    chart = alt.Chart(df).mark_tick().encode(
        #x='hoursminutes(Stamp):T'
        x=alt.X('Stamp:T', title='Timestamp', scale=x_scale,axis=alt.Axis(format='%H:%M')), #,axis=alt.Axis(values=custom_x_ticks)), # axis=alt.Axis(values=custom_x_ticks)),
        y=alt.Y('line:O', title='Line', scale=y_scale), 
        color=alt.value('red')  # Apply the color encoding
    ).properties(
        title=f'Production line {line} testing data in the {timeofday} ({date})',#,#{beacon}',
        width=400,  
        height=25
    ).interactive()

    return chart

#date_graph = day+ "-" + month + "-2023"
plots = 1
if plots:
    factory1_morning = strip_factory_plot(df_factory1_morn, date_year, line = 'A', timeofday = 'morning')
    factory1_afternoon = strip_factory_plot(df_factory1_after, date_year, line = 'A', timeofday = 'afternoon')

    factory2_morning = strip_factory_plot(df_factory1_morn, date_year, line = 'B', timeofday = 'morning')
    factory2_afternoon = strip_factory_plot(df_factory1_after, date_year, line = 'B', timeofday = 'afternoon')

    beacon1_morning = strip_plot(df1_morning, date_year,beacon = 'A', timeofday = 'morning')
    beacon1_afternoon = strip_plot(df1_afternoon,date_year, beacon = 'A', timeofday = 'afternoon')

    beacon2_morning = strip_plot(df2_morning, date_year, beacon = 'B', timeofday = 'morning')
    beacon2_afternoon = strip_plot(df2_afternoon, date_year,beacon = 'B', timeofday = 'afternoon')

    beacon1_factory_morn = alt.vconcat(factory1_morning, beacon1_morning).configure_tick( bandSize=25)
    beacon1_factory_after = alt.vconcat(factory1_afternoon, beacon1_afternoon).configure_tick( bandSize=25)
    #beacon1 = alt.hconcat(beacon1_factory_morn,beacon1_factory_after).configure_tick( bandSize=25)
    beacon1_factory_morn.display()
    beacon1_factory_after.display()

    beacon2_factory_morn = alt.vconcat(factory2_morning, beacon2_morning).configure_tick( bandSize=25)
    beacon2_factory_after = alt.vconcat(factory2_afternoon, beacon2_afternoon).configure_tick( bandSize=25)
    beacon2_factory_morn.display()
    beacon2_factory_after.display()
#beacon2 = alt.hconcat(beacon2_factory_morn,beacon2_factory_after).configure_tick( bandSize=25)
#beacon2.display()



#beacon1 = alt.hconcat(beacon1_morning, beacon1_afternoon)#.configure_tick(     bandSize=25)
#beacon2 = alt.hconcat(beacon2_morning, beacon2_afternoon)#.configure_tick(    bandSize=25)


alt.VConcatChart(...)

alt.VConcatChart(...)

alt.VConcatChart(...)

alt.VConcatChart(...)

In [ ]:
print("Beacon 1")
#time_in_beacon1 =  time_in_station_ble(df1_morning)
visits1 = count_visits_station(df1_morning,1)
dwell1 = count_visits_station(df1_morning,2)
#display(df_rssi1)

print("Beacon 2")
#time_in_beacon2 = time_in_station_ble(df2_morning)
visits2 = count_visits_station(df2_morning,1)
dwell2 = count_visits_station(df2_morning,2)

products1 = len(df_factory1_morn)
products2 = len(df_factory2_morn)
print(products1)
print(products2)

In [ ]:
start_date = '06/10/2023' #'03/10/2023'
end_date = '06/10/2023' # '10/10/2023'

start_date = pd.to_datetime(start_date, format='%d/%m/%Y')
end_date = pd.to_datetime(end_date, format='%d/%m/%Y')

# Create a date range
date_range = pd.date_range(start=start_date, end=end_date)

columns = ['Date','Beacon', 'Time in Station', 'Station Counts', 'Average Dwell Times']
df_metrics = pd.DataFrame(columns=columns)

def extract_time_for_station(row, station):
    times = row['Time in Station']
    return times[station - 1]

def extract_station_counts(row, station):
    times = row['Station Counts']
    return times[station - 1]

def extract_average_dwell_times(row, station):
    times = row['Average Dwell Times']
    return times[station - 1]

for date in date_range:

    day = str(date.day).zfill(2)
    month = str(date.month).zfill(2)
    date_ = day + "_" + month
    print(date_)
    df1 = process_file(2,date_)
    if df1.empty:
        print("No data in this date")
        continue
    df2 = process_file(1,date_)
    df3 = process_file(3,date_)

    df_factory1 = df_factory[(df_factory['Line'] == 1)]
    df_factory2 = df_factory[(df_factory['Line'] == 2)]
    start_time = pd.to_datetime(date_year + " 09:00:00")
    end_time = pd.to_datetime(date_year + " 18:00:00")
    df_factory1 = df_factory1[((df_factory1['Stamp'] >= start_time) & (df_factory1['Stamp'] <= end_time))].reset_index(drop = True)
    df_factory2 = df_factory2[((df_factory2['Stamp'] >= start_time) & (df_factory2['Stamp'] <= end_time))].reset_index(drop = True)

    df_rssi1 = analyse_df(df1,df2,df3, beacon = 1) #beacon 1
    df_rssi2 = analyse_df(df1,df2,df3, beacon = 2) #beacon 2
    

    ref_values=np.array(
    [[-66.44549266, -74.26545582, -73.88842253],
    [-60.86128482, -43.43985976, -65.93448369],
    [-68.73783186, -56.43292456, -61.49354318]])

    df_rssi1 = classify_points(df_rssi1,best_weights,ref_values)
    df_rssi2 = classify_points(df_rssi2,best_weights,ref_values)    

    df_rssi1 = remove_outlier_transition(df_rssi1)
    df_rssi1 = remove_outliers(df_rssi1)
    df_rssi1 = remove_outliers2(df_rssi1)
    df_rssi1 = remove_outliers3(df_rssi1)
    df_rssi1 = remove_outliers4(df_rssi1)
    df_rssi1 = remove_outliers3(df_rssi1)
    df_rssi1 = remove_outliers2(df_rssi1)
    df_rssi1 = remove_outliers(df_rssi1)

    df_rssi2 = remove_outlier_transition(df_rssi2)
    df_rssi2 = remove_outliers(df_rssi2)
    df_rssi2 = remove_outliers2(df_rssi2)
    df_rssi2 = remove_outliers3(df_rssi2)
    df_rssi2 = remove_outliers4(df_rssi2)
    df_rssi2 = remove_outliers3(df_rssi2)
    df_rssi2 = remove_outliers2(df_rssi2)
    df_rssi2 = remove_outliers(df_rssi2)

    df_rssi1_ = remove_breaks_from_rssi(df_rssi1, df_factory1)
    df_rssi2_ = remove_breaks_from_rssi(df_rssi2, df_factory2)
    print(df_factory1)
    print(df_rssi1_)

    
    df1_morning_ = df_rssi1[((df_rssi1_['ts'] >= start_time) & (df_rssi1_['ts'] <= start_lunch))]
    df1_afternoon_ = df_rssi1[((df_rssi1_['ts'] >= end_lunch) & (df_rssi1_['ts'] <= end_time))]
    #print(df1_morning_)
    df1_morning = resample_df(df1_morning_,freq = '3S')
    #print(df1_morning)
    df1_afternoon = resample_df(df1_afternoon_,freq = '3S')
    #df1_morning = df1_morning.reset_index(drop=True)
    #df1_afternoon = df1_afternoon.reset_index(drop=True)


    df2_morning_ = df_rssi2_[((df_rssi2['ts'] >= start_time) & (df_rssi2_['ts'] <= start_lunch))] 
    df2_afternoon_ = df_rssi2_[((df_rssi2['ts'] >= end_lunch) & (df_rssi2_['ts'] <= end_time))]

    df2_morning = resample_df(df2_morning_,freq = '3S')
    df2_afternoon = resample_df(df2_afternoon_,freq = '3S')
    df2_morning = df2_morning.reset_index(drop=True)
    df2_afternoon = df2_afternoon.reset_index(drop=True )


    df_factory1_morn = df_factory1[((df_factory1['Stamp'] >= start_time) & (df_factory1['Stamp'] <= start_lunch))] 
    df_factory1_after = df_factory1[((df_factory1['Stamp'] >= end_lunch) & (df_factory1['Stamp'] <= end_time))]
    df_factory1_morn = df_factory1_morn.reset_index(drop=True )
    df_factory1_after = df_factory1_after.reset_index(drop=True )


    df_factory2_morn = df_factory2[((df_factory2['Stamp'] >= start_time) & (df_factory2['Stamp'] <= start_lunch))] 
    df_factory2_after = df_factory2[((df_factory2['Stamp'] >= end_lunch) & (df_factory2['Stamp'] <= end_time))]
    df_factory2_morn = df_factory2_morn.reset_index(drop=True )
    df_factory2_after = df_factory2_after.reset_index(drop=True )



    # Iterate through a range of dates and calculate metrics
    time_in_station1_morn = time_in_station_ble(df1_morning)
    station_counts1_m  = count_visits_station(df1_morning,1)
    average_dwell_times1_m = count_visits_station(df1_morning,2)

    time_in_station1_after = time_in_station_ble(df1_afternoon)
    station_counts1_a = count_visits_station(df1_afternoon,1)
    average_dwell_times1_a = count_visits_station(df1_afternoon,2)

    time_in_station2_morn =time_in_station_ble(df2_morning)
    station_counts2_m  = count_visits_station(df2_morning,1)
    average_dwell_times2_m  = count_visits_station(df2_morning,2)

    time_in_station2_after = time_in_station_ble(df2_afternoon)
    station_counts2_a = count_visits_station(df2_afternoon, 1)
    average_dwell_times2_a= count_visits_station(df2_afternoon,2)

    # Combine metrics for morning and afternoon
    combined_time_in_station1 = [((morn_value + after_value) / 2) for morn_value, after_value in zip(time_in_station1_morn, time_in_station1_after)]
    combined_station_counts1 = [((morn_value + after_value) / 2) for morn_value, after_value in zip(station_counts1_m, station_counts1_a)]
    combined_average_dwell_times1 = [((morn_value + after_value) / 2) for morn_value, after_value in zip(average_dwell_times1_m, average_dwell_times1_a)]

    combined_time_in_station2 = [((morn_value + after_value) / 2) for morn_value, after_value in zip(time_in_station2_morn, time_in_station2_after)]
    combined_station_counts2 = [((morn_value + after_value) / 2) for morn_value, after_value in zip(station_counts2_m, station_counts2_a)]
    combined_average_dwell_times2 = [((morn_value + after_value) / 2) for morn_value, after_value in zip(average_dwell_times2_m, average_dwell_times2_a)]

    # Add the metrics to the DataFrame
    data1 = {
        'Beacon': 1,
        'Time in Station': combined_time_in_station1,
        'Station Counts': combined_station_counts1,
        'Average Dwell Times': combined_average_dwell_times1
    }
    data2 = {
    'Beacon': 2, 
    'Time in Station': combined_time_in_station2, 
    'Station Counts': combined_station_counts2, 
    'Average Dwell Times': combined_average_dwell_times2
    }
    df1 = pd.DataFrame([data1])
    df2 = pd.DataFrame([data2])
    df_metrics = pd.concat([df_metrics,df1], ignore_index=True)
    df_metrics = pd.concat([df_metrics,df2], ignore_index=True)

    display(df_metrics)
    for station in range(1, 4):  # Assuming you have stations from 1 to 3
        df_metrics[f'Time in Station{station}'] = df_metrics.apply(extract_time_for_station, station=station, axis=1)

    
    for station in range(1, 4):  # Assuming you have stations from 1 to 3
        df_metrics[f'Station Counts{station}'] = df_metrics.apply(extract_station_counts, station=station, axis=1)
        
    # Calculate and add the average dwell times for each station
    for station in range(1, 4):  # Assuming you have stations from 1 to 3
        df_metrics[f'Average Dwell Times{station}'] = df_metrics.apply(extract_average_dwell_times, station=station, axis=1)

    df_metrics = df_metrics.drop(columns=['Time in Station', 'Station Counts', 'Average Dwell Times'])

    df_metrics['Date'] = date_
display(df_metrics)


#averages = df.groupby('Beacon').mean()

    

In [55]:
#24 07
def strip_plot_24(df):
    df['ts'] = df['ts'].astype(str)  
    y_scale = alt.Scale(domain=[1.0, 2.0, 3.0])
    color_real = alt.Color('real station:N', scale=alt.Scale(
        domain=[1.0, 2.0, 3.0], range=['cornflowerblue', 'darkorange','violet'])
            , legend=alt.Legend(title='Station'))
    color = alt.Color('station:N', scale=alt.Scale(
        domain=[1.0, 2.0, 3.0], range=['gray', 'cornflowerblue', 'darkorange','violet'])
            , legend=alt.Legend(title='Station'))

    custom_x_ticks = ['2023-08-24 17:58:00', '2023-08-24 18:00:00', '2023-08-24 18:02:00', '2023-08-24 18:04:00', '2023-08-24 18:06:00', '2023-08-24 18:08:00', '2023-08-24 18:10:00']    #1 inicio 17:57:30 -> 18:01:00

    #1 inicio 17:57:30 -> 18:01:00
    # fim 18:02:00 -> 18:06:00
    #3 monta cargas 18:07:00 -> 18:10:00
    chart_stat = alt.Chart(df).mark_tick().encode(
        
        x=alt.X('ts:T', title='Timestamp',axis=alt.Axis(format='%H:%M')),
        y=alt.Y('station:O', title='Station', scale=y_scale),
        color=color  # Apply the color encoding
    ).properties(
        title=f'Predicted station of beacon',#{beacon}',
        width=400,  # Set the width to 600 pixels (adjust as needed)
        height=100  # Set the height to 400 pixels (adjust as needed)
    )
    
    chart_real = alt.Chart(df).mark_tick().encode(
        x=alt.X('ts:T', title='Timestamp',axis=alt.Axis(format='%H:%M')),
        y=alt.Y('real station:O', title='Station', scale=y_scale),
        color=color_real  # Apply the color encoding
    ).properties(
        title=f'Real station of beacon',#{beacon}',
        width=400,  # Set the width to 600 pixels (adjust as needed)
        height=100  # Set the height to 400 pixels (adjust as needed)
    )
    chart = alt.vconcat(chart_real,chart_stat).configure_tick(bandSize=25)

    return chart.interactive()

if date == "24_08":
    chart = strip_plot_24(df24_08)#.configure_tick(bandSize=20)
    chart.display()

#display(df_rssi2_resample)
    
#chart1 = strip_plot(df_rssi1_,beacon=1)
#chart2 = strip_plot(df_rssi2_resample.loc[df_rssi2_resample['real station'] != 0],include_3=1)

#chart1 | 
#display(df_rssi2_resample[df_rssi2_resample['real station'] != 0])
#chart2